# * VINSIGHT : SubBase, 60DPD, Reported SubBase
    กลุ่ม KPIs ของ Postpaid Reported SubBase B2C : TRUE มาจากพี่เอ้รัน scripts พี่หนุ่ย (SubBase, 60DPD, Gain/Loss 60DPD)
    กลุ่ม KPIs ของ Postpaid Reported SubBase B2C : DTAC ยังมาจากพี่เอกรันให้อยู่ (SubBase, 60DPD, Gain/Loss 60DPD)
    กลุ่ม KPIs ของ FTTx Reported SubBase ยังมาจากพี่เอกรันให้อยู่ (SubBase, 60DPD, Gain/Loss 60DPD)
    Postpaid Reported SubBase B2C เริ่มตั้งแต่ June2024++
    Prepaid Active Caller : TMH เริ่มตั้งแต่ May2024++

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000702' --Prepaid SubBase
        , 'TB1S000702' --Prepaid SubBase : TMH
        , 'DB1S000702' --Prepaid SubBase : DTAC
        , 'B1S000700' --Prepaid Reported SubBase
        , 'TB1S000700' --Prepaid Reported SubBase : TMH
        , 'DB1S000700' --Prepaid Reported SubBase : DTAC
        
        , 'B2S010602' --Postpaid SubBase B2C
        , 'DB2S010602' --Postpaid SubBase B2C : DTAC
        , 'TB2S010602' --Postpaid SubBase B2C : TMH
        , 'B2S010601' --Postpaid 60DPD B2C
        , 'TB2S010601' --Postpaid 60DPD B2C : TMH
        , 'DB2S010601' --Postpaid 60DPD B2C : DTAC
        , 'B2S010603' --Postpaid Gain/Loss 60DPD B2C
        , 'TB2S010603' --Postpaid Gain/Loss 60DPD B2C : TMH
        , 'DB2S010603' --Postpaid Gain/Loss 60DPD B2C : DTAC
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
        , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
        
        , 'TB3S020603' --FTTx 60DPD
        , 'TB3S020606' --FTTx Gain/Loss 60DPD
        , 'TB3S000600' --FTTx Reported SubBase
        , 'TB3S020604' --FTTx SubBase
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-10-27, 12:50:07

DataFrame: 12635 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

In [4]:
# v_metric_list = [
#     'B1S000702' #Prepaid SubBase
#     , 'TB1S000702' #Prepaid SubBase : TMH
#     , 'DB1S000702' #Prepaid SubBase : DTAC
# 	, 'B1S000700' #Prepaid Reported SubBase
# 	, 'TB1S000700' #Prepaid Reported SubBase : TMH
# 	, 'DB1S000700' #Prepaid Reported SubBase : DTAC

# 	, 'B2S010602' #Postpaid SubBase B2C
#     , 'TB2S010602' #Postpaid SubBase B2C : TMH
# 	, 'DB2S010602' #Postpaid SubBase B2C : DTAC
# 	, 'B2S010601' #Postpaid 60DPD B2C
#     , 'TB2S010601' #Postpaid 60DPD B2C : TMH
# 	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC
# 	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
#     , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
# 	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC
# 	, 'B2S010600' #Postpaid Reported SubBase B2C
#     , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
# 	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
	
#     , 'TB3S020604' #FTTx SubBase
# 	, 'TB3S020603' #FTTx 60DPD
# 	, 'TB3S020606' #FTTx Gain/Loss 60DPD
# 	, 'TB3S000600' #FTTx Reported SubBase
#     ]

## Prepaid by Period

In [5]:
''' Prepaid : SubBase Yearly '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
pre_sub_yearly_df = pre_sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
pre_sub_yearly_df = pre_sub_yearly_df.loc[pre_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_yearly_df['PRE_SB'] = np.where(pre_sub_yearly_df['METRIC_CD']=='B1S000702', pre_sub_yearly_df['P'], 0)
pre_sub_yearly_df['PRE_SB_T'] = np.where(pre_sub_yearly_df['METRIC_CD']=='TB1S000702', pre_sub_yearly_df['P'], 0)
pre_sub_yearly_df['PRE_SB_D'] = np.where(pre_sub_yearly_df['METRIC_CD']=='DB1S000702', pre_sub_yearly_df['P'], 0)

pre_sub_yearly_df['PRE_RPT_SB'] = np.where(pre_sub_yearly_df['METRIC_CD']=='B1S000700', pre_sub_yearly_df['P'], 0)
pre_sub_yearly_df['PRE_RPT_SB_T'] = np.where(pre_sub_yearly_df['METRIC_CD']=='TB1S000700', pre_sub_yearly_df['P'], 0)
pre_sub_yearly_df['PRE_RPT_SB_D'] = np.where(pre_sub_yearly_df['METRIC_CD']=='DB1S000700', pre_sub_yearly_df['P'], 0)

pre_sub_yearly_df = pre_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_yearly_df = pre_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
pre_sub_yearly_df = pre_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_yearly_df_display = pre_sub_yearly_df.copy()
pre_sub_yearly_df_display['ACTUAL_AS_OF'] = pre_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = pre_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_yearly_df_display[col] = pre_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,2024,2025-09-22 06:15:44,20241231,0,"24,782,672",0,"34,713,745","21,220,694","13,493,051"
1,2025,2025-10-27 06:22:35,20251025,0,"21,961,303",0,"31,933,511","19,123,900","12,809,611"


In [6]:
''' Prepaid : SubBase Quarterly '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
pre_sub_quarterly_df = pre_sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
pre_sub_quarterly_df = pre_sub_quarterly_df.loc[pre_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_quarterly_df['PRE_SB'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='B1S000702', pre_sub_quarterly_df['P'], 0)
pre_sub_quarterly_df['PRE_SB_T'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='TB1S000702', pre_sub_quarterly_df['P'], 0)
pre_sub_quarterly_df['PRE_SB_D'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='DB1S000702', pre_sub_quarterly_df['P'], 0)

pre_sub_quarterly_df['PRE_RPT_SB'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='B1S000700', pre_sub_quarterly_df['P'], 0)
pre_sub_quarterly_df['PRE_RPT_SB_T'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='TB1S000700', pre_sub_quarterly_df['P'], 0)
pre_sub_quarterly_df['PRE_RPT_SB_D'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='DB1S000700', pre_sub_quarterly_df['P'], 0)

pre_sub_quarterly_df = pre_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_quarterly_df = pre_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
pre_sub_quarterly_df = pre_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_quarterly_df_display = pre_sub_quarterly_df.copy()
pre_sub_quarterly_df_display['ACTUAL_AS_OF'] = pre_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = pre_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_quarterly_df_display[col] = pre_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,20241,2025-09-22 06:15:44,20240331,0,"25,748,719",0,"35,788,546","21,409,544","14,379,002"
1,20242,2025-09-22 06:15:44,20240630,0,"25,384,868",0,"35,251,969","20,914,621","14,337,348"
2,20243,2025-09-22 06:15:44,20240930,0,"24,977,745",0,"34,127,734","20,682,878","13,444,856"
3,20244,2025-09-22 06:15:44,20241231,0,"24,782,672",0,"34,713,745","21,220,694","13,493,051"
4,20251,2025-10-27 06:22:35,20250331,0,"24,578,672",0,"33,574,556","20,647,169","12,927,387"
5,20252,2025-10-27 06:22:35,20250630,0,"22,959,730",0,"32,408,494","19,640,822","12,767,672"
6,20253,2025-10-27 06:22:35,20250930,0,"21,760,938",0,"31,881,030","19,000,466","12,880,564"
7,20254,2025-10-27 06:22:35,20251025,0,"21,961,303",0,"31,933,511","19,123,900","12,809,611"


In [7]:
''' Prepaid : SubBase Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
pre_sub_monthly_df = pre_sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
pre_sub_monthly_df = pre_sub_monthly_df.loc[pre_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_monthly_df['PRE_SB'] = np.where(pre_sub_monthly_df['METRIC_CD']=='B1S000702', pre_sub_monthly_df['P'], 0)
pre_sub_monthly_df['PRE_SB_T'] = np.where(pre_sub_monthly_df['METRIC_CD']=='TB1S000702', pre_sub_monthly_df['P'], 0)
pre_sub_monthly_df['PRE_SB_D'] = np.where(pre_sub_monthly_df['METRIC_CD']=='DB1S000702', pre_sub_monthly_df['P'], 0)

pre_sub_monthly_df['PRE_RPT_SB'] = np.where(pre_sub_monthly_df['METRIC_CD']=='B1S000700', pre_sub_monthly_df['P'], 0)
pre_sub_monthly_df['PRE_RPT_SB_T'] = np.where(pre_sub_monthly_df['METRIC_CD']=='TB1S000700', pre_sub_monthly_df['P'], 0)
pre_sub_monthly_df['PRE_RPT_SB_D'] = np.where(pre_sub_monthly_df['METRIC_CD']=='DB1S000700', pre_sub_monthly_df['P'], 0)

pre_sub_monthly_df = pre_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_monthly_df = pre_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
pre_sub_monthly_df = pre_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_monthly_df_display = pre_sub_monthly_df.copy()
pre_sub_monthly_df_display['ACTUAL_AS_OF'] = pre_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = pre_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_monthly_df_display[col] = pre_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,202401,2025-09-22 06:15:44,20240131,0,0,0,0,0,"14,487,788"
1,202402,2025-09-22 06:15:44,20240229,0,"25,840,436",0,"36,446,445","21,794,228","14,652,217"
2,202403,2025-09-22 06:15:44,20240331,0,"25,748,719",0,"35,788,546","21,409,544","14,379,002"
3,202404,2025-09-22 06:15:44,20240430,0,"25,446,381",0,"35,394,485","21,026,743","14,367,742"
4,202405,2025-09-22 06:15:44,20240531,0,"25,329,884",0,"35,466,641","20,992,491","14,474,150"
5,202406,2025-09-22 06:15:44,20240630,0,"25,384,868",0,"35,251,969","20,914,621","14,337,348"
6,202407,2025-09-22 06:15:44,20240731,0,"25,008,838",0,"35,014,325","20,935,553","14,078,772"
7,202408,2025-09-22 06:15:44,20240831,0,"25,090,909",0,"34,574,792","20,868,927","13,705,865"
8,202409,2025-09-22 06:15:44,20240930,0,"24,977,745",0,"34,127,734","20,682,878","13,444,856"
9,202410,2025-09-22 06:15:44,20241031,0,"24,836,634",0,"33,895,674","20,579,008","13,316,666"


In [8]:
''' Prepaid : SubBase Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
pre_sub_weekly_df = pre_sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
pre_sub_weekly_df = pre_sub_weekly_df.loc[pre_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_weekly_df['PRE_SB'] = np.where(pre_sub_weekly_df['METRIC_CD']=='B1S000702', pre_sub_weekly_df['P'], 0)
pre_sub_weekly_df['PRE_SB_T'] = np.where(pre_sub_weekly_df['METRIC_CD']=='TB1S000702', pre_sub_weekly_df['P'], 0)
pre_sub_weekly_df['PRE_SB_D'] = np.where(pre_sub_weekly_df['METRIC_CD']=='DB1S000702', pre_sub_weekly_df['P'], 0)

pre_sub_weekly_df['PRE_RPT_SB'] = np.where(pre_sub_weekly_df['METRIC_CD']=='B1S000700', pre_sub_weekly_df['P'], 0)
pre_sub_weekly_df['PRE_RPT_SB_T'] = np.where(pre_sub_weekly_df['METRIC_CD']=='TB1S000700', pre_sub_weekly_df['P'], 0)
pre_sub_weekly_df['PRE_RPT_SB_D'] = np.where(pre_sub_weekly_df['METRIC_CD']=='DB1S000700', pre_sub_weekly_df['P'], 0)

pre_sub_weekly_df = pre_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_weekly_df = pre_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
pre_sub_weekly_df = pre_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_weekly_df_display = pre_sub_weekly_df.copy()
pre_sub_weekly_df_display['ACTUAL_AS_OF'] = pre_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = pre_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_weekly_df_display[col] = pre_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,2025036,2025-10-27 06:22:35,20250907,0,"21,533,778",0,"31,887,621","19,023,042","12,864,579"
1,2025037,2025-10-27 06:22:35,20250914,0,"21,594,142",0,"31,876,626","19,002,115","12,874,511"
2,2025038,2025-10-27 06:22:35,20250921,0,"21,645,869",0,"31,853,408","18,976,152","12,877,256"
3,2025039,2025-10-27 06:22:35,20250928,0,"21,753,590",0,"31,881,368","18,999,694","12,881,674"
4,2025040,2025-10-27 06:22:35,20251005,0,"21,802,151",0,"31,948,776","19,068,405","12,880,371"
5,2025041,2025-10-27 06:22:35,20251012,0,"21,848,807",0,"31,957,703","19,084,349","12,873,354"
6,2025042,2025-10-27 06:22:35,20251019,0,"21,914,977",0,"31,983,551","19,116,806","12,866,745"
7,2025043,2025-10-27 06:22:35,20251025,0,"21,961,303",0,"31,933,511","19,123,900","12,809,611"


In [42]:
''' Prepaid : SubBase Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
pre_sub_daily_df = pre_sub_daily_df.loc[pre_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_daily_df['PRE_SB'] = np.where(pre_sub_daily_df['METRIC_CD']=='B1S000702', pre_sub_daily_df['P'], 0)
pre_sub_daily_df['PRE_SB_T'] = np.where(pre_sub_daily_df['METRIC_CD']=='TB1S000702', pre_sub_daily_df['P'], 0)
pre_sub_daily_df['PRE_SB_D'] = np.where(pre_sub_daily_df['METRIC_CD']=='DB1S000702', pre_sub_daily_df['P'], 0)

pre_sub_daily_df['PRE_RPT_SB'] = np.where(pre_sub_daily_df['METRIC_CD']=='B1S000700', pre_sub_daily_df['P'], 0)
pre_sub_daily_df['PRE_RPT_SB_T'] = np.where(pre_sub_daily_df['METRIC_CD']=='TB1S000700', pre_sub_daily_df['P'], 0)
pre_sub_daily_df['PRE_RPT_SB_D'] = np.where(pre_sub_daily_df['METRIC_CD']=='DB1S000700', pre_sub_daily_df['P'], 0)

pre_sub_daily_df = pre_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_daily_df = pre_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
pre_sub_daily_df = pre_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_daily_df_display = pre_sub_daily_df.copy()
mod_col_list = pre_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_daily_df_display[col] = pre_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,202510,20251026,2025-10-27 06:22:35,0,0,0,0,0,0
1,202510,20251025,2025-10-27 06:22:35,0,0,0,"31,933,511","19,123,900","12,809,611"
2,202510,20251024,2025-10-27 06:22:35,0,"21,961,303",0,"31,954,816","19,128,887","12,825,929"
3,202510,20251023,2025-10-27 06:22:35,0,"21,951,338",0,"31,963,627","19,127,825","12,835,802"
4,202510,20251022,2025-10-27 06:22:35,0,"21,941,542",0,"31,971,569","19,126,246","12,845,323"
5,202510,20251021,2025-10-27 06:22:35,0,"21,938,130",0,"31,985,162","19,129,170","12,855,992"
6,202510,20251020,2025-10-27 06:22:35,0,"21,928,706",0,"31,992,522","19,126,923","12,865,599"
7,202510,20251019,2025-10-27 06:22:35,0,"21,914,977",0,"31,983,551","19,116,806","12,866,745"
8,202510,20251018,2025-10-27 06:22:35,0,"21,898,623",0,"31,980,826","19,115,197","12,865,629"
9,202510,20251017,2025-10-27 06:22:35,0,"21,885,326",0,"31,968,035","19,105,628","12,862,407"


## Postpaid B2C by Period

In [10]:
''' Postpaid B2C : SubBase Yearly '''

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
post_b2c_sub_yearly_df = post_b2c_sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
post_b2c_sub_yearly_df = post_b2c_sub_yearly_df.loc[post_b2c_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_yearly_df['POST_SB_B2C'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='B2S010602', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='TB2S010602', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='DB2S010602', post_b2c_sub_yearly_df['P'], 0)

post_b2c_sub_yearly_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='B2S010601', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='TB2S010601', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='DB2S010601', post_b2c_sub_yearly_df['P'], 0)

post_b2c_sub_yearly_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='B2S010603', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='TB2S010603', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='DB2S010603', post_b2c_sub_yearly_df['P'], 0)

post_b2c_sub_yearly_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='B2S010600', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='TB2S010600', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='DB2S010600', post_b2c_sub_yearly_df['P'], 0)

post_b2c_sub_yearly_df = post_b2c_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_yearly_df = post_b2c_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
post_b2c_sub_yearly_df = post_b2c_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                               , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_yearly_df_display = post_b2c_sub_yearly_df.copy()
post_b2c_sub_yearly_df_display['ACTUAL_AS_OF'] = post_b2c_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = post_b2c_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_yearly_df_display[col] = post_b2c_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_GL_60DPD_B2C,POST_GL_60DPD_B2C_T,POST_GL_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,2024,2025-09-22 06:15:44,20241231,"14,287,442","9,102,725","5,184,717","208,886","80,142","128,744",0,"-6,627","-1,412","14,158,698","9,102,725","5,055,973"
1,2025,2025-10-27 06:22:35,20251025,"14,144,271","9,160,873","4,983,433","218,490","109,856","108,634",0,"9,840","-1,262","13,940,780","9,051,017","4,879,958"


In [11]:
''' Postpaid B2C : SubBase Quarterly '''

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df.loc[post_b2c_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_quarterly_df['POST_SB_B2C'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='B2S010602', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='TB2S010602', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='DB2S010602', post_b2c_sub_quarterly_df['P'], 0)

post_b2c_sub_quarterly_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='B2S010601', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='TB2S010601', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='DB2S010601', post_b2c_sub_quarterly_df['P'], 0)

post_b2c_sub_quarterly_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='B2S010603', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='TB2S010603', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='DB2S010603', post_b2c_sub_quarterly_df['P'], 0)

post_b2c_sub_quarterly_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='B2S010600', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='TB2S010600', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='DB2S010600', post_b2c_sub_quarterly_df['P'], 0)

post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                               , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_quarterly_df_display = post_b2c_sub_quarterly_df.copy()
post_b2c_sub_quarterly_df_display['ACTUAL_AS_OF'] = post_b2c_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = post_b2c_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_quarterly_df_display[col] = post_b2c_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_GL_60DPD_B2C,POST_GL_60DPD_B2C_T,POST_GL_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,20241,2025-09-22 06:15:44,20240331,0,"8,994,853",0,0,"79,598",0,0,"-18,781",0,0,"8,994,853",0
1,20242,2025-09-22 06:15:44,20240630,"14,391,657","9,042,184","5,349,473","247,217","81,727","165,490",0,"5,476","-1,222","14,226,167","9,042,184","5,183,983"
2,20243,2025-09-22 06:15:44,20240930,"14,338,027","9,068,470","5,269,557","237,190","85,778","151,412",0,-275,"-2,237","14,186,615","9,068,470","5,118,145"
3,20244,2025-09-22 06:15:44,20241231,"14,287,442","9,102,725","5,184,717","208,886","80,142","128,744",0,"-6,627","-1,412","14,158,698","9,102,725","5,055,973"
4,20251,2025-10-27 06:22:35,20250331,"14,288,650","9,170,605","5,118,045","203,838","83,785","120,053",0,"-4,018","5,360","14,084,812","9,086,820","4,997,992"
5,20252,2025-10-27 06:22:35,20250630,"14,257,825","9,179,698","5,078,127","221,285","95,691","125,594",0,"-3,251","-1,699","14,036,540","9,084,007","4,952,533"
6,20253,2025-10-27 06:22:35,20250930,"14,157,676","9,152,437","5,005,239","204,996","91,388","113,608",0,"-2,536","-1,767","13,952,680","9,061,049","4,891,631"
7,20254,2025-10-27 06:22:35,20251025,"14,144,271","9,160,873","4,983,433","218,490","109,856","108,634",0,"9,840","-1,262","13,940,780","9,051,017","4,879,958"


In [12]:
''' Postpaid B2C : SubBase Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
post_b2c_sub_monthly_df = post_b2c_sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
post_b2c_sub_monthly_df = post_b2c_sub_monthly_df.loc[post_b2c_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_monthly_df['POST_SB_B2C'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='B2S010602', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='TB2S010602', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='DB2S010602', post_b2c_sub_monthly_df['P'], 0)

post_b2c_sub_monthly_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='B2S010601', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='TB2S010601', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='DB2S010601', post_b2c_sub_monthly_df['P'], 0)

post_b2c_sub_monthly_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='B2S010603', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='TB2S010603', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='DB2S010603', post_b2c_sub_monthly_df['P'], 0)

post_b2c_sub_monthly_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='B2S010600', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='TB2S010600', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='DB2S010600', post_b2c_sub_monthly_df['P'], 0)

post_b2c_sub_monthly_df = post_b2c_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_monthly_df = post_b2c_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
post_b2c_sub_monthly_df = post_b2c_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                               , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_monthly_df_display = post_b2c_sub_monthly_df.copy()
post_b2c_sub_monthly_df_display['ACTUAL_AS_OF'] = post_b2c_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = post_b2c_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_monthly_df_display[col] = post_b2c_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_GL_60DPD_B2C,POST_GL_60DPD_B2C_T,POST_GL_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,202402,2025-09-22 06:15:44,20240229,0,"8,998,482",0,0,"98,379",0,0,"-2,417",0,0,"8,998,482",0
1,202403,2025-09-22 06:15:44,20240331,0,"8,994,853",0,0,"79,598",0,0,"-18,781",0,0,"8,994,853",0
2,202404,2025-09-22 06:15:44,20240430,0,"8,996,646",0,0,"74,373",0,0,"-5,225",0,0,"8,996,646",0
3,202405,2025-09-22 06:15:44,20240531,0,"9,023,324",0,0,"76,251",0,0,"1,878",0,0,"9,023,324",0
4,202406,2025-09-22 06:15:44,20240630,"14,391,657","9,042,184","5,349,473","247,217","81,727","165,490",0,"5,476","-1,222","14,226,167","9,042,184","5,183,983"
5,202407,2025-09-22 06:15:44,20240731,"14,372,506","9,052,569","5,319,937","247,415","83,942","163,473",0,"2,215","-1,492","14,209,033","9,052,569","5,156,464"
6,202408,2025-09-22 06:15:44,20240831,"14,353,346","9,061,002","5,292,344","248,968","86,053","162,915",0,"2,111","-1,534","14,190,431","9,061,002","5,129,429"
7,202409,2025-09-22 06:15:44,20240930,"14,338,027","9,068,470","5,269,557","237,190","85,778","151,412",0,-275,"-2,237","14,186,615","9,068,470","5,118,145"
8,202410,2025-09-22 06:15:44,20241031,"14,321,392","9,080,143","5,241,249","227,616","85,630","141,986",0,-148,"-1,973","14,179,406","9,080,143","5,099,263"
9,202411,2025-09-22 06:15:44,20241130,"14,293,575","9,088,656","5,204,919","219,822","86,769","133,053",0,"1,139","-1,219","14,160,522","9,088,656","5,071,866"


In [13]:
''' Postpaid B2C : SubBase Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
post_b2c_sub_weekly_df = post_b2c_sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
post_b2c_sub_weekly_df = post_b2c_sub_weekly_df.loc[post_b2c_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_weekly_df['POST_SB_B2C'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='B2S010602', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='TB2S010602', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='DB2S010602', post_b2c_sub_weekly_df['P'], 0)

post_b2c_sub_weekly_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='B2S010601', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='TB2S010601', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='DB2S010601', post_b2c_sub_weekly_df['P'], 0)

post_b2c_sub_weekly_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='B2S010603', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='TB2S010603', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='DB2S010603', post_b2c_sub_weekly_df['P'], 0)

post_b2c_sub_weekly_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='B2S010600', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='TB2S010600', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='DB2S010600', post_b2c_sub_weekly_df['P'], 0)

post_b2c_sub_weekly_df = post_b2c_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_weekly_df = post_b2c_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
post_b2c_sub_weekly_df = post_b2c_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                               , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_weekly_df_display = post_b2c_sub_weekly_df.copy()
post_b2c_sub_weekly_df_display['ACTUAL_AS_OF'] = post_b2c_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = post_b2c_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_weekly_df_display[col] = post_b2c_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_GL_60DPD_B2C,POST_GL_60DPD_B2C_T,POST_GL_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,2025036,2025-10-27 06:22:35,20250907,"14,184,015","9,159,049","5,024,966","207,072","89,114","117,958",0,"-2,361","-1,048","13,976,943","9,069,935","4,907,008"
1,2025037,2025-10-27 06:22:35,20250914,"14,179,264","9,159,895","5,019,369","219,550","99,562","119,988",0,"7,780","6,084","13,959,714","9,060,333","4,899,381"
2,2025038,2025-10-27 06:22:35,20250921,"14,175,876","9,158,791","5,017,085","219,641","99,282","120,359",0,"-3,439",-645,"13,956,235","9,059,509","4,896,726"
3,2025039,2025-10-27 06:22:35,20250928,"14,160,860","9,152,108","5,008,752","215,816","96,668","119,148",0,"-3,380","4,574","13,945,044","9,055,440","4,889,604"
4,2025040,2025-10-27 06:22:35,20251005,"14,158,131","9,155,593","5,002,538","204,193","91,871","112,322",0,"-1,412",-743,"13,953,938","9,063,722","4,890,216"
5,2025041,2025-10-27 06:22:35,20251012,"14,153,581","9,157,772","4,995,809","215,862","103,837","112,025",0,"6,723","-1,327","13,937,719","9,053,935","4,883,784"
6,2025042,2025-10-27 06:22:35,20251019,"14,146,325","9,158,748","4,987,577","216,622","103,061","113,561",0,"-2,158",-942,"13,929,703","9,055,687","4,874,016"
7,2025043,2025-10-27 06:22:35,20251025,"14,144,271","9,160,873","4,983,433","218,490","109,856","108,634",0,"9,840","-1,262","13,940,780","9,051,017","4,879,958"


In [14]:
''' Postpaid B2C : SubBase Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
post_b2c_sub_daily_df = post_b2c_sub_daily_df.loc[post_b2c_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_daily_df['POST_SB_B2C'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='B2S010602', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='TB2S010602', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='DB2S010602', post_b2c_sub_daily_df['P'], 0)

post_b2c_sub_daily_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='B2S010601', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='TB2S010601', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='DB2S010601', post_b2c_sub_daily_df['P'], 0)

post_b2c_sub_daily_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='B2S010603', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='TB2S010603', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='DB2S010603', post_b2c_sub_daily_df['P'], 0)

post_b2c_sub_daily_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='B2S010600', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='TB2S010600', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='DB2S010600', post_b2c_sub_daily_df['P'], 0)

post_b2c_sub_daily_df = post_b2c_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_daily_df = post_b2c_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
post_b2c_sub_daily_df = post_b2c_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                               , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_daily_df_display = post_b2c_sub_daily_df.copy()
mod_col_list = post_b2c_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_daily_df_display[col] = post_b2c_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_GL_60DPD_B2C,POST_GL_60DPD_B2C_T,POST_GL_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,202510,20251026,2025-10-27 06:22:35,0,0,0,0,0,0,0,0,0,0,0,0
1,202510,20251025,2025-10-27 06:22:35,0,"9,160,873",0,"218,490","109,856","108,634",0,"9,840",0,0,"9,051,017",0
2,202510,20251024,2025-10-27 06:22:35,"14,144,271","9,160,838","4,983,433","203,491","100,016","103,475",0,"-2,104","-1,262","13,940,780","9,060,822","4,879,958"
3,202510,20251023,2025-10-27 06:22:35,"14,143,978","9,160,221","4,983,757","207,193","102,120","105,073",0,"-3,045",-603,"13,936,785","9,058,101","4,878,684"
4,202510,20251022,2025-10-27 06:22:35,"14,143,472","9,160,494","4,982,978","210,943","105,165","105,778",0,"-1,102","-4,353","13,932,529","9,055,329","4,877,200"
5,202510,20251021,2025-10-27 06:22:35,"14,146,167","9,159,295","4,986,872","216,954","106,267","110,687",0,"6,363",-886,"13,929,213","9,053,028","4,876,185"
6,202510,20251020,2025-10-27 06:22:35,"14,146,346","9,159,051","4,987,295","211,456","99,904","111,552",0,"-3,157","-1,804","13,934,890","9,059,147","4,875,743"
7,202510,20251019,2025-10-27 06:22:35,"14,146,325","9,158,748","4,987,577","216,622","103,061","113,561",0,"-2,158",-942,"13,929,703","9,055,687","4,874,016"
8,202510,20251018,2025-10-27 06:22:35,"14,144,791","9,157,424","4,987,367","219,785","105,219","114,566",0,"6,148","4,682","13,925,006","9,052,205","4,872,801"
9,202510,20251017,2025-10-27 06:22:35,"14,148,358","9,157,053","4,991,305","209,412","99,071","110,341",0,"-4,201",-875,"13,938,946","9,057,982","4,880,964"


## FTTx by Period

In [15]:
''' FTTx : SubBase Yearly '''

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
fttx_sub_yearly_df = fttx_sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
fttx_sub_yearly_df = fttx_sub_yearly_df.loc[fttx_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_yearly_df['FTTX_SB'] = np.where(fttx_sub_yearly_df['METRIC_CD']=='TB3S020604', fttx_sub_yearly_df['P'], 0)
fttx_sub_yearly_df['FTTX_60DPD'] = np.where(fttx_sub_yearly_df['METRIC_CD']=='TB3S020603', fttx_sub_yearly_df['P'], 0)
fttx_sub_yearly_df['FTTX_GL_60DPD'] = np.where(fttx_sub_yearly_df['METRIC_CD']=='TB3S020606', fttx_sub_yearly_df['P'], 0)
fttx_sub_yearly_df['FTTX_RPT_SB'] = np.where(fttx_sub_yearly_df['METRIC_CD']=='TB3S000600', fttx_sub_yearly_df['P'], 0)

fttx_sub_yearly_df = fttx_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_yearly_df = fttx_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
fttx_sub_yearly_df = fttx_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_yearly_df_display = fttx_sub_yearly_df.copy()
fttx_sub_yearly_df_display['ACTUAL_AS_OF'] = fttx_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = fttx_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_yearly_df_display[col] = fttx_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,2024,2025-09-22 06:15:44,20241231,"3,270,700","115,811","6,834","3,154,889"
1,2025,2025-10-27 06:22:35,20251024,"3,321,208","88,435","3,057","3,232,773"


In [16]:
''' FTTx : SubBase Quarterly '''

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
fttx_sub_quarterly_df = fttx_sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
fttx_sub_quarterly_df = fttx_sub_quarterly_df.loc[fttx_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_quarterly_df['FTTX_SB'] = np.where(fttx_sub_quarterly_df['METRIC_CD']=='TB3S020604', fttx_sub_quarterly_df['P'], 0)
fttx_sub_quarterly_df['FTTX_60DPD'] = np.where(fttx_sub_quarterly_df['METRIC_CD']=='TB3S020603', fttx_sub_quarterly_df['P'], 0)
fttx_sub_quarterly_df['FTTX_GL_60DPD'] = np.where(fttx_sub_quarterly_df['METRIC_CD']=='TB3S020606', fttx_sub_quarterly_df['P'], 0)
fttx_sub_quarterly_df['FTTX_RPT_SB'] = np.where(fttx_sub_quarterly_df['METRIC_CD']=='TB3S000600', fttx_sub_quarterly_df['P'], 0)

fttx_sub_quarterly_df = fttx_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_quarterly_df = fttx_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
fttx_sub_quarterly_df = fttx_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_quarterly_df_display = fttx_sub_quarterly_df.copy()
fttx_sub_quarterly_df_display['ACTUAL_AS_OF'] = fttx_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = fttx_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_quarterly_df_display[col] = fttx_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,20241,2025-09-22 06:15:44,20240331,"3,200,919","110,632",649,"3,090,287"
1,20242,2025-09-22 06:15:44,20240630,"3,219,954","112,870",636,"3,107,084"
2,20243,2025-09-22 06:15:44,20240930,"3,246,868","119,564","2,863","3,127,304"
3,20244,2025-09-22 06:15:44,20241231,"3,270,700","115,811","6,834","3,154,889"
4,20251,2025-10-27 06:22:35,20250331,"3,305,177","117,460","2,510","3,187,717"
5,20252,2025-10-27 06:22:35,20250630,"3,304,012","96,071","2,697","3,207,941"
6,20253,2025-10-27 06:22:35,20250930,"3,318,426","92,582","2,973","3,225,844"
7,20254,2025-10-27 06:22:35,20251024,"3,321,208","88,435","3,057","3,232,773"


In [17]:
''' FTTx : SubBase Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
fttx_sub_monthly_df = fttx_sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
fttx_sub_monthly_df = fttx_sub_monthly_df.loc[fttx_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_monthly_df['FTTX_SB'] = np.where(fttx_sub_monthly_df['METRIC_CD']=='TB3S020604', fttx_sub_monthly_df['P'], 0)
fttx_sub_monthly_df['FTTX_60DPD'] = np.where(fttx_sub_monthly_df['METRIC_CD']=='TB3S020603', fttx_sub_monthly_df['P'], 0)
fttx_sub_monthly_df['FTTX_GL_60DPD'] = np.where(fttx_sub_monthly_df['METRIC_CD']=='TB3S020606', fttx_sub_monthly_df['P'], 0)
fttx_sub_monthly_df['FTTX_RPT_SB'] = np.where(fttx_sub_monthly_df['METRIC_CD']=='TB3S000600', fttx_sub_monthly_df['P'], 0)

fttx_sub_monthly_df = fttx_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_monthly_df = fttx_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
fttx_sub_monthly_df = fttx_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_monthly_df_display = fttx_sub_monthly_df.copy()
fttx_sub_monthly_df_display['ACTUAL_AS_OF'] = fttx_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = fttx_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_monthly_df_display[col] = fttx_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,202402,2025-09-22 06:15:44,20240229,"3,196,974","112,709","1,759","3,084,265"
1,202403,2025-09-22 06:15:44,20240331,"3,200,919","110,632",649,"3,090,287"
2,202404,2025-09-22 06:15:44,20240430,"3,202,928","111,161","-1,855","3,091,767"
3,202405,2025-09-22 06:15:44,20240531,"3,209,626","111,112","2,194","3,098,514"
4,202406,2025-09-22 06:15:44,20240630,"3,219,954","112,870",636,"3,107,084"
5,202407,2025-09-22 06:15:44,20240731,"3,229,133","114,298",959,"3,114,835"
6,202408,2025-09-22 06:15:44,20240831,"3,239,197","117,168",985,"3,122,029"
7,202409,2025-09-22 06:15:44,20240930,"3,246,868","119,564","2,863","3,127,304"
8,202410,2025-09-22 06:15:44,20241031,"3,255,234","120,867","-1,414","3,134,367"
9,202411,2025-09-22 06:15:44,20241130,"3,262,940","124,095","1,240","3,138,845"


In [18]:
''' FTTx : SubBase Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
fttx_sub_weekly_df = fttx_sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_sub_weekly_df = fttx_sub_weekly_df.loc[fttx_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_weekly_df['FTTX_SB'] = np.where(fttx_sub_weekly_df['METRIC_CD']=='TB3S020604', fttx_sub_weekly_df['P'], 0)
fttx_sub_weekly_df['FTTX_60DPD'] = np.where(fttx_sub_weekly_df['METRIC_CD']=='TB3S020603', fttx_sub_weekly_df['P'], 0)
fttx_sub_weekly_df['FTTX_GL_60DPD'] = np.where(fttx_sub_weekly_df['METRIC_CD']=='TB3S020606', fttx_sub_weekly_df['P'], 0)
fttx_sub_weekly_df['FTTX_RPT_SB'] = np.where(fttx_sub_weekly_df['METRIC_CD']=='TB3S000600', fttx_sub_weekly_df['P'], 0)

fttx_sub_weekly_df = fttx_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_weekly_df = fttx_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
fttx_sub_weekly_df = fttx_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_weekly_df_display = fttx_sub_weekly_df.copy()
fttx_sub_weekly_df_display['ACTUAL_AS_OF'] = fttx_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = fttx_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_weekly_df_display[col] = fttx_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,2025036,2025-10-27 06:22:35,20250907,"3,318,295","100,299",532,"3,217,996"
1,2025037,2025-10-27 06:22:35,20250914,"3,317,665","101,284","-4,256","3,216,381"
2,2025038,2025-10-27 06:22:35,20250921,"3,318,785","100,110",665,"3,218,675"
3,2025039,2025-10-27 06:22:35,20250928,"3,315,873","96,104","4,251","3,219,769"
4,2025040,2025-10-27 06:22:35,20251005,"3,317,153","90,104","7,349","3,227,049"
5,2025041,2025-10-27 06:22:35,20251012,"3,320,622","94,099","-3,685","3,226,523"
6,2025042,2025-10-27 06:22:35,20251019,"3,322,478","93,319",692,"3,229,159"
7,2025043,2025-10-27 06:22:35,20251024,"3,321,208","88,435","3,057","3,232,773"


In [43]:
''' FTTx : SubBase Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_sub_daily_df = fttx_sub_daily_df.loc[fttx_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_daily_df['FTTX_SB'] = np.where(fttx_sub_daily_df['METRIC_CD']=='TB3S020604', fttx_sub_daily_df['P'], 0)
fttx_sub_daily_df['FTTX_60DPD'] = np.where(fttx_sub_daily_df['METRIC_CD']=='TB3S020603', fttx_sub_daily_df['P'], 0)
fttx_sub_daily_df['FTTX_GL_60DPD'] = np.where(fttx_sub_daily_df['METRIC_CD']=='TB3S020606', fttx_sub_daily_df['P'], 0)
fttx_sub_daily_df['FTTX_RPT_SB'] = np.where(fttx_sub_daily_df['METRIC_CD']=='TB3S000600', fttx_sub_daily_df['P'], 0)

fttx_sub_daily_df = fttx_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_daily_df = fttx_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
fttx_sub_daily_df = fttx_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_daily_df_display = fttx_sub_daily_df.copy()
mod_col_list = fttx_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_daily_df_display[col] = fttx_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,202510,20251026,2025-10-27 06:22:35,0,0,0,0
1,202510,20251025,2025-10-27 06:22:35,0,0,0,0
2,202510,20251024,2025-10-27 06:22:35,"3,321,208","88,435","3,057","3,232,773"
3,202510,20251023,2025-10-27 06:22:35,"3,323,128","91,492",521,"3,231,636"
4,202510,20251022,2025-10-27 06:22:35,"3,322,186","92,013","4,479","3,230,173"
5,202510,20251021,2025-10-27 06:22:35,"3,324,793","96,492","-3,891","3,228,301"
6,202510,20251020,2025-10-27 06:22:35,"3,323,691","92,601",718,"3,231,090"
7,202510,20251019,2025-10-27 06:22:35,"3,322,478","93,319",692,"3,229,159"
8,202510,20251018,2025-10-27 06:22:35,"3,321,344","94,011","-4,240","3,227,333"
9,202510,20251017,2025-10-27 06:22:35,"3,320,105","89,771","2,673","3,230,334"


## Products Summary

### Prep Monthly Data

In [20]:
''' Monthly Summary '''

monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
monthly_df_display['ACTUAL_AS_OF'] = monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [21]:
''' Parameter '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202501

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202510


### Prepaid

In [22]:
''' Prepaid SubBase '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
    ]

prepaid_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_subbase_df = prepaid_subbase_df.loc[prepaid_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_subbase_df = prepaid_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20251024,0,"21,961,303","21,828,397","21,827,959","21,827,959","-21,961,303"


In [23]:
''' Prepaid Reported SubBase '''

v_metric_list = [
    'B1S000700' #Prepaid Reported SubBase
    , 'TB1S000700' #Prepaid Reported SubBase : TMH
    , 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

prepaid_reported_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_reported_subbase_df = prepaid_reported_subbase_df.loc[prepaid_reported_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_reported_subbase_df = prepaid_reported_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_reported_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,Prepaid,B1S000700,Prepaid Reported SubBase,2025-10-27 06:22:35,20251025,0,"31,933,511","31,779,890","31,779,890","31,779,890","-31,933,511"
1,202510,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-10-27 06:22:35,20251025,0,"12,809,611","12,793,607","12,793,607","12,793,607","-12,809,611"
2,202510,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-10-27 06:22:35,20251025,0,"19,123,900","18,986,283","18,986,283","18,986,283","-19,123,900"


### Postpaid B2C

In [24]:
''' Postpaid SubBase B2C '''

v_metric_list = [
    'B2S010602' #Postpaid SubBase B2C
    , 'DB2S010602' #Postpaid SubBase B2C : DTAC
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
    ]

postpaid_subbase_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.loc[postpaid_subbase_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_subbase_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,Postpaid,B2S010602,Postpaid SubBase B2C,2025-10-27 06:22:35,20251024,0,"14,144,271","14,138,381","14,138,373","14,138,373","-14,144,271"
1,202510,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-10-27 06:22:35,20251024,0,"4,983,433","4,977,785","4,977,785","4,977,785","-4,983,433"
2,202510,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,20251025,0,"9,160,873","9,160,633","9,160,625","9,160,625","-9,160,873"


In [25]:
''' Postpaid 60DPD B2C '''

v_metric_list = [
    'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
    , 'DB2S010601' #Postpaid 60DPD B2C : DTAC
    ]

postpaid_60dpd_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_60dpd_b2c_df = postpaid_60dpd_b2c_df.loc[postpaid_60dpd_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_60dpd_b2c_df = postpaid_60dpd_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_60dpd_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,Postpaid,B2S010601,Postpaid 60DPD B2C,2025-10-27 06:22:35,20251025,0,"218,490","218,424","218,424","218,424","-218,490"
1,202510,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-10-27 06:22:35,20251025,0,"108,634","108,570","108,570","108,570","-108,634"
2,202510,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,20251025,0,"109,856","109,854","109,854","109,854","-109,856"


In [26]:
''' Postpaid Gain/Loss 60DPD B2C '''

v_metric_list = [
    'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
    , 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC
    ]

postpaid_gl_60dpd_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_gl_60dpd_b2c_df = postpaid_gl_60dpd_b2c_df.loc[postpaid_gl_60dpd_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_gl_60dpd_b2c_df = postpaid_gl_60dpd_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_gl_60dpd_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,Postpaid,DB2S010603,Postpaid Gain/Loss 60DPD B2C : DTAC,2025-10-27 06:22:35,20251024,"-1,262","-1,262","-1,262","-1,262","-1,262",0
1,202510,Postpaid,TB2S010603,Postpaid Gain/Loss 60DPD B2C : TMH,2025-10-27 06:22:35,20251025,"9,840","9,840","9,840","9,840","9,840",0


In [27]:
''' Postpaid Reported SubBase B2C '''

v_metric_list = [
    'B2S010600' #Postpaid Reported SubBase B2C
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    ]

postpaid_reported_subbase_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_reported_subbase_b2c_df = postpaid_reported_subbase_b2c_df.loc[postpaid_reported_subbase_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_reported_subbase_b2c_df = postpaid_reported_subbase_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_reported_subbase_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-10-27 06:22:35,20251024,0,"13,940,780","13,934,956","13,934,948","13,934,948","-13,940,780"
1,202510,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20251024,0,"4,879,958","4,874,374","4,874,374","4,874,374","-4,879,958"
2,202510,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,20251025,0,"9,051,017","9,050,779","9,050,771","9,050,771","-9,051,017"


### FTTx

In [28]:
''' FTTx SubBase '''

v_metric_list = [
    'TB3S020604' #FTTx SubBase
    ]

fttx_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_subbase_df = fttx_subbase_df.loc[fttx_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_subbase_df = fttx_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20251024,0,"3,321,208","3,321,129","3,321,129","3,321,129","-3,321,208"


In [29]:
''' FTTx 60DPD '''

v_metric_list = [
    'TB3S020603' #FTTx 60DPD
    ]

fttx_60dpd_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_60dpd_df = fttx_60dpd_df.loc[fttx_60dpd_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_60dpd_df = fttx_60dpd_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_60dpd_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,TOL,TB3S020603,FTTx 60DPD,2025-10-27 06:22:35,20251024,0,"88,435","88,435","88,435","88,435","-88,435"


In [30]:
''' FTTx Gain/Loss 60DPD '''

v_metric_list = [
    'TB3S020606' #FTTx Gain/Loss 60DPD
    ]

fttx_gl_60dpd_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_gl_60dpd_df = fttx_gl_60dpd_df.loc[fttx_gl_60dpd_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_gl_60dpd_df = fttx_gl_60dpd_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_gl_60dpd_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,TOL,TB3S020606,FTTx Gain/Loss 60DPD,2025-10-27 06:22:35,20251024,0,"3,057","3,057","3,057","3,057","-3,057"


In [31]:
''' FTTx Reported SubBase '''

v_metric_list = [
    'TB3S000600' #FTTx Reported SubBase
    ]

fttx_reported_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_reported_subbase_df = fttx_reported_subbase_df.loc[fttx_reported_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_reported_subbase_df = fttx_reported_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_reported_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202510,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20251024,0,"3,232,773","3,232,694","3,232,694","3,232,694","-3,232,773"


## ** Current Issue

In [32]:
''' Monthly : Prepaid SubBase : DTAC '''

v_metric_cd = 'DB1S000702'

issue_prepaid_subbase_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_subbase_dtac_df = issue_prepaid_subbase_dtac_df.loc[issue_prepaid_subbase_dtac_df['TM_KEY_MTH']>=202401]
issue_prepaid_subbase_dtac_df = issue_prepaid_subbase_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_prepaid_subbase_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [33]:
''' Monthly : Prepaid Reported SubBase : DTAC '''

v_metric_cd = 'DB1S000700'

issue_prepaid_reported_subbase_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_reported_subbase_dtac_df = issue_prepaid_reported_subbase_dtac_df.loc[issue_prepaid_reported_subbase_dtac_df['TM_KEY_MTH']>=202401]
issue_prepaid_reported_subbase_dtac_df = issue_prepaid_reported_subbase_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_prepaid_reported_subbase_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202401,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240131,0,"14,487,788","14,467,869","14,467,869","14,467,869","-14,487,788"
1,202402,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240229,0,"14,652,217","14,630,204","14,630,204","14,630,204","-14,652,217"
2,202403,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240331,0,"14,379,002","14,354,466","14,354,466","14,354,466","-14,379,002"
3,202404,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240430,0,"14,367,742","14,343,827","14,343,827","14,343,827","-14,367,742"
4,202405,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240531,0,"14,474,150","14,453,687","14,453,687","14,453,687","-14,474,150"
5,202406,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240630,0,"14,337,348","14,320,485","14,320,485","14,320,485","-14,337,348"
6,202407,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240731,0,"14,078,772","14,060,619","14,060,619","14,060,619","-14,078,772"
7,202408,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240831,0,"13,705,865","13,687,654","13,687,654","13,687,654","-13,705,865"
8,202409,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20240930,0,"13,444,856","13,427,824","13,427,824","13,427,824","-13,444,856"
9,202410,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-09-22 06:15:44,20241031,0,"13,316,666","13,298,454","13,298,454","13,298,454","-13,316,666"


In [34]:
''' Monthly : Postpaid SubBase B2C : TMH '''

v_metric_cd = 'TB2S010602'

issue_postpaid_subbase_b2c_true_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_subbase_b2c_true_df = issue_postpaid_subbase_b2c_true_df.loc[issue_postpaid_subbase_b2c_true_df['TM_KEY_MTH']>=202401]
issue_postpaid_subbase_b2c_true_df = issue_postpaid_subbase_b2c_true_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_subbase_b2c_true_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20240229,0,"8,998,482","8,998,152","8,998,139","8,998,139","-8,998,482"
1,202403,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20240331,0,"8,994,853","8,987,179","8,987,169","8,987,169","-8,994,853"
2,202404,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20240430,0,"8,996,646","8,989,810","8,989,800","8,989,800","-8,996,646"
3,202405,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20240531,0,"9,023,324","9,009,568","9,009,558","9,009,558","-9,023,324"
4,202406,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20240630,0,"9,042,184","9,041,896","9,041,886","9,041,886","-9,042,184"
5,202407,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20240731,0,"9,052,569","9,052,314","9,052,304","9,052,304","-9,052,569"
6,202408,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20240831,0,"9,061,002","9,060,761","9,060,751","9,060,751","-9,061,002"
7,202409,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20240930,0,"9,068,470","9,068,233","9,068,223","9,068,223","-9,068,470"
8,202410,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20241031,0,"9,080,143","9,079,912","9,079,902","9,079,902","-9,080,143"
9,202411,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-09-22 06:15:44,20241130,0,"9,088,656","9,088,441","9,088,431","9,088,431","-9,088,656"


In [35]:
''' Daily : Postpaid SubBase B2C : TMH '''

v_metict_cd = 'TB2S010602'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_post_sb_b2c_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_sb_b2c_tmh_df = issue_daily_post_sb_b2c_tmh_df.loc[issue_daily_post_sb_b2c_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_post_sb_b2c_tmh_df = issue_daily_post_sb_b2c_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_sb_b2c_tmh_df_display = issue_daily_post_sb_b2c_tmh_df.copy()
mod_col_list = issue_daily_post_sb_b2c_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_sb_b2c_tmh_df_display[col] = issue_daily_post_sb_b2c_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_sb_b2c_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,160,873","9,160,633","9,160,625","9,160,625"
2,202510,20251024,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,160,838","9,160,596","9,160,588","9,160,588"
3,202510,20251023,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,160,221","9,159,979","9,159,971","9,159,971"
4,202510,20251022,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,160,494","9,160,254","9,160,246","9,160,246"
5,202510,20251021,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,159,295","9,159,056","9,159,048","9,159,048"
6,202510,20251020,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,159,051","9,158,813","9,158,805","9,158,805"
7,202510,20251019,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,158,748","9,158,510","9,158,502","9,158,502"
8,202510,20251018,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,157,424","9,157,186","9,157,178","9,157,178"
9,202510,20251017,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,"9,157,053","9,151,813","9,151,804","9,151,804"


In [36]:
''' Monthly : Postpaid SubBase B2C : DTAC '''

v_metric_cd = 'DB2S010602'

issue_postpaid_subbase_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_subbase_b2c_dtac_df = issue_postpaid_subbase_b2c_dtac_df.loc[issue_postpaid_subbase_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_subbase_b2c_dtac_df = issue_postpaid_subbase_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_subbase_b2c_dtac_df = issue_postpaid_subbase_b2c_dtac_df.drop(['C', 'DIFF (C-P)'], axis=1)
issue_postpaid_subbase_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P,G,H,HH
0,202406,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-09-22 06:15:44,20240630,"5,349,473","5,336,373","5,154,102","5,336,373"
1,202407,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-09-22 06:15:44,20240731,"5,319,937","5,306,861","5,126,018","5,306,861"
2,202408,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-09-22 06:15:44,20240831,"5,292,344","5,279,274","5,098,221","5,279,274"
3,202409,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-09-22 06:15:44,20240930,"5,269,557","5,256,503","5,076,200","5,256,503"
4,202410,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-09-22 06:15:44,20241031,"5,241,249","5,228,437","5,049,529","5,228,437"
5,202411,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-09-22 06:15:44,20241130,"5,204,919","5,199,257","5,020,614","5,199,257"
6,202412,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-09-22 06:15:44,20241231,"5,184,717","5,179,054","5,000,605","5,179,054"
7,202501,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-10-27 06:22:35,20250131,"5,161,625","5,155,964","4,979,737","5,155,964"
8,202502,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-10-27 06:22:35,20250228,"5,138,667","5,133,010","5,133,010","5,133,010"
9,202503,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-10-27 06:22:35,20250331,"5,118,045","5,112,387","5,112,387","5,112,387"


In [37]:
''' Monthly : Postpaid 60DPD B2C : TMH '''

v_metric_cd = 'TB2S010601'

issue_postpaid_60dpd_b2c_true_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_60dpd_b2c_true_df = issue_postpaid_60dpd_b2c_true_df.loc[issue_postpaid_60dpd_b2c_true_df['TM_KEY_MTH']>=202401]
issue_postpaid_60dpd_b2c_true_df = issue_postpaid_60dpd_b2c_true_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_60dpd_b2c_true_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20240228,0,"98,379","98,376","98,376","98,376","-98,379"
1,202403,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20240331,0,"79,598","79,593","79,593","79,593","-79,598"
2,202404,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20240430,0,"74,373","74,372","74,372","74,372","-74,373"
3,202405,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20240531,0,"76,251","76,251","76,251","76,251","-76,251"
4,202406,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20240630,0,"81,727","81,725","81,725","81,725","-81,727"
5,202407,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20240731,0,"83,942","83,940","83,940","83,940","-83,942"
6,202408,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20240831,0,"86,053","86,053","86,053","86,053","-86,053"
7,202409,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20240930,0,"85,778","85,777","85,777","85,777","-85,778"
8,202410,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20241031,0,"85,630","85,630","85,630","85,630","-85,630"
9,202411,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-09-22 06:15:44,20241130,0,"86,769","86,769","86,769","86,769","-86,769"


In [38]:
''' Daily : Postpaid 60DPD B2C : TMH '''

v_metict_cd = 'TB2S010601'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_post_60dpd_b2c_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_60dpd_b2c_tmh_df = issue_daily_post_60dpd_b2c_tmh_df.loc[issue_daily_post_60dpd_b2c_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_post_60dpd_b2c_tmh_df = issue_daily_post_60dpd_b2c_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_60dpd_b2c_tmh_df_display = issue_daily_post_60dpd_b2c_tmh_df.copy()
mod_col_list = issue_daily_post_60dpd_b2c_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_60dpd_b2c_tmh_df_display[col] = issue_daily_post_60dpd_b2c_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_60dpd_b2c_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"109,856","109,854","109,854","109,854"
2,202510,20251024,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"100,016","100,014","100,014","100,014"
3,202510,20251023,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"102,120","102,117","102,117","102,117"
4,202510,20251022,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"105,165","105,162","105,162","105,162"
5,202510,20251021,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"106,267","106,264","106,264","106,264"
6,202510,20251020,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"99,904","99,901","99,901","99,901"
7,202510,20251019,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"103,061","103,058","103,058","103,058"
8,202510,20251018,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"105,219","105,216","105,216","105,216"
9,202510,20251017,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-10-27 06:22:35,"99,071","99,068","99,068","99,068"


In [39]:
''' Monthly : Postpaid 60DPD B2C : DTAC '''

v_metric_cd = 'DB2S010601'

issue_postpaid_60dpd_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_60dpd_b2c_dtac_df = issue_postpaid_60dpd_b2c_dtac_df.loc[issue_postpaid_60dpd_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_60dpd_b2c_dtac_df = issue_postpaid_60dpd_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_60dpd_b2c_dtac_df = issue_postpaid_60dpd_b2c_dtac_df.drop(['C', 'DIFF (C-P)'], axis=1)
issue_postpaid_60dpd_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P,G,H,HH
0,202406,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-09-22 06:15:44,20240630,"165,490","165,409","161,834","165,409"
1,202407,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-09-22 06:15:44,20240731,"163,473","163,393","159,897","163,393"
2,202408,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-09-22 06:15:44,20240831,"162,915","162,834","159,379","162,834"
3,202409,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-09-22 06:15:44,20240930,"151,412","151,333","148,074","151,333"
4,202410,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-09-22 06:15:44,20241031,"141,986","141,901","138,754","141,901"
5,202411,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-09-22 06:15:44,20241130,"133,053","132,985","129,965","132,985"
6,202412,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-09-22 06:15:44,20241231,"128,744","128,678","125,779","128,678"
7,202501,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-10-27 06:22:35,20250131,"127,686","127,622","124,809","127,622"
8,202502,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-10-27 06:22:35,20250228,"124,747","124,684","124,684","124,684"
9,202503,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-10-27 06:22:35,20250331,"120,053","119,988","119,988","119,988"


In [40]:
''' Monthly : Postpaid Reported SubBase B2C : TMH '''

v_metric_cd = 'TB2S010600'

issue_postpaid_reported_subbase_b2c_true_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_reported_subbase_b2c_true_df = issue_postpaid_reported_subbase_b2c_true_df.loc[issue_postpaid_reported_subbase_b2c_true_df['TM_KEY_MTH']>=202401]
issue_postpaid_reported_subbase_b2c_true_df = issue_postpaid_reported_subbase_b2c_true_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_reported_subbase_b2c_true_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240229,0,"8,998,482","8,998,152","8,998,139","8,998,139","-8,998,482"
1,202403,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240331,0,"8,994,853","8,987,179","8,987,169","8,987,169","-8,994,853"
2,202404,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240430,0,"8,996,646","8,989,810","8,989,800","8,989,800","-8,996,646"
3,202405,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240531,0,"9,023,324","9,009,568","9,009,558","9,009,558","-9,023,324"
4,202406,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240630,0,"9,042,184","9,041,896","9,041,886","9,041,886","-9,042,184"
5,202407,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240731,0,"9,052,569","9,052,314","9,052,304","9,052,304","-9,052,569"
6,202408,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240831,0,"9,061,002","9,060,761","9,060,751","9,060,751","-9,061,002"
7,202409,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240930,0,"9,068,470","9,068,233","9,068,223","9,068,223","-9,068,470"
8,202410,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20241031,0,"9,080,143","9,079,912","9,079,902","9,079,902","-9,080,143"
9,202411,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20241130,0,"9,088,656","9,088,441","9,088,431","9,088,431","-9,088,656"


In [41]:
''' Monthly : Postpaid Reported SubBase B2C : DTAC '''

v_metric_cd = 'DB2S010600'

issue_postpaid_reported_subbase_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_reported_subbase_b2c_dtac_df = issue_postpaid_reported_subbase_b2c_dtac_df.loc[issue_postpaid_reported_subbase_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_reported_subbase_b2c_dtac_df = issue_postpaid_reported_subbase_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_reported_subbase_b2c_dtac_df = issue_postpaid_reported_subbase_b2c_dtac_df.drop(['C', 'DIFF (C-P)'], axis=1)
issue_postpaid_reported_subbase_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P,G,H,HH
0,202406,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20240630,"5,183,983","5,170,964","4,992,268","5,170,964"
1,202407,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20240731,"5,156,464","5,143,468","4,966,121","5,143,468"
2,202408,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20240831,"5,129,429","5,116,440","4,938,842","5,116,440"
3,202409,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20240930,"5,118,145","5,105,170","4,928,126","5,105,170"
4,202410,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20241031,"5,099,263","5,086,536","4,910,775","5,086,536"
5,202411,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20241130,"5,071,866","5,066,272","4,890,649","5,066,272"
6,202412,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20241231,"5,055,973","5,050,376","4,874,826","5,050,376"
7,202501,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250131,"5,033,939","5,028,342","4,854,928","5,028,342"
8,202502,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250228,"5,013,920","5,008,326","5,008,326","5,008,326"
9,202503,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250331,"4,997,992","4,992,399","4,992,399","4,992,399"
